In [2]:
import pandas as pd
import numpy as np
import re, os, pickle
from os.path import join
from tqdm import tqdm
from datetime import datetime

# 옛날거 반영해야 하므로 전산에서 다운 받는다.
# 필요컬럼은 채무자키,계좌키,합의서체결일,채무구분
# 주민번호는 비식별처리되어있으며, 계좌별로 합의서체결일이 다를 수 있으므로 계좌키로 작업한다.

account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}

In [7]:
# 파일읽기 #####################################################################################################
wd = r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\시효\신복실효건 반영하기"
credit = pd.read_excel(join(wd, "231228 신용회복새창_20231228_1305.xlsx"), dtype=str)[["채무자키","계좌키","합의서체결일","채무구분"]]

# 최근 자산정리에서 파일 읽기
# 전산수정을 위해 raw_data 아닌 계좌 읽기 pkl vs xlsx
dataset_dir = r"D:\3.자산\전산 dataset\솔림\231221\pkl"
filename_account = "계좌조회새창_20231221_1841"
filename_raw_data = "2.솔림_전체_원데이터_231221"

account = pd.read_pickle(join(dataset_dir, filename_account + ".pkl"))[['채무자키','계좌키','시효완성일','시효연장사유','시효중단여부']]
# account = pd.read_excel(r"D:\3.자산\전산 dataset\솔림\231221\계좌조회새창_20231221_1841.xlsx", dtype=account_dtype)[['채무자키','계좌키','시효완성일','시효연장사유','시효중단여부']]

# raw_data 읽기 : 채권상태 확인
raw_data = pd.read_pickle(join(dataset_dir, filename_raw_data + ".pkl"))[['계좌키','채권상태','새채무상태']]
################################################################################################################

In [13]:
# 합의서체결일자 없는 거, 보증채무 지우기
# credit.합의서체결일[0]
print(len(credit))
credit.drop(credit[credit.합의서체결일.isna()].index, axis=0, inplace=True)
credit.drop(credit[credit.채무구분=="보증채무"].index, axis=0, inplace=True)
print(len(credit))

19909
19909


In [44]:
# 신복에서 시효완성일 및 시효연장사유 작성
credit["시효완성일"] = (pd.to_numeric(credit.합의서체결일.str[:4]) + 5).astype(str) +credit.합의서체결일.str[4:]
credit["시효연장사유"] = "신용회복 합의서 체결:"+credit.합의서체결일

# 계좌와 병합
merged = credit.merge(account, on=['채무자키','계좌키'], how='left', suffixes=['_신복','_계좌'])
# raw_data와 병합
merged = merged.merge(raw_data, on=["계좌키"], how="left")

# 시효완성일 역순정렬 후 중복계좌 드랍하기
print("계좌키 중복제거 전 : ", len(merged))
merged = merged.sort_values(['시효완성일_신복'], ascending=False)
merged = merged.drop_duplicates(['계좌키'], keep='first', ignore_index=True)
print("계좌키 중복제거 후 : ", len(merged))

# 희한하게 지금은 삭제한 채무자키,계좌키를 쓰는 건이 4건있다.(계좌번호는 현재도 확인되므로 키값만 수정한듯) - merge대상에서 삭제
merged = merged.dropna(subset=["새채무상태"],how='any', axis=0)
merged = merged.reset_index(drop=True)
print("na값 제거 후 :", len(merged))

계좌키 중복제거 전 :  19909
계좌키 중복제거 후 :  19909
na값 제거 후 : 19905


In [47]:

# 시효수정할 것만 추리기 : 중단 아니고, 전산 시효가 느린 거
시효수정_상태반영 = merged.query('~새채무상태.str.contains("개인회생|신용회복|종료")')
시효수정_중단아님 = 시효수정_상태반영.query('시효중단여부 != "Y"')
시효수정 = 시효수정_중단아님.query('시효완성일_신복 > 시효완성일_계좌')
print(f"종료,조정건 제외 후 : {len(시효수정_상태반영)}, 중단아닌 거 : {len(시효수정_중단아님)}, 시효수정할 것 : {len(시효수정)}")


종료,조정건 제외 후 : 1215, 중단아닌 거 : 1215, 시효수정할 것 : 0


In [6]:
시효수정.rename(columns={'시효완성일_신복':'최초시효완성일', '시효연장사유_신복':'최초시효완성일메모'}, inplace=True)
###########################################################################
시효수정.to_excel(join(wd, "231030 신용회복 합의서체결일반영 시효수정_비교파일.xlsx"), index=False)
###########################################################################

#### 전산 수정 작업
# 1. 시효 만료, 임박건만 할지, 전체를 할지 선택
# 2. 전산에서 종결건 제외 후 엑셀일괄수정 > 계좌정보